In [11]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms

# ---------- Parámetros de la meta ----------
total_calories = 500 * 7  # calorías totales objetivo (semana)
percentage_prot = 0.3
percentage_carb = 0.5
percentage_fat = 0.2

cal_prot = round(percentage_prot * total_calories)
cal_carb = round(percentage_carb * total_calories)
cal_fat = round(percentage_fat * total_calories)
print("calorias de proteina: ", cal_prot, "calorias de carb: ", cal_carb, "calorias de grasa: ", cal_fat)

prot_cal_p_gram = 4
carb_cal_p_gram = 4
fat_cal_p_gram = 9

gram_prot = cal_prot / prot_cal_p_gram
gram_carb = cal_carb / carb_cal_p_gram
gram_fat = cal_fat / fat_cal_p_gram
print("gramos de proteina: ", gram_prot, "gramos de carb: ", gram_carb, "gramos de grasa: ", gram_fat)

# ---------- Tabla de productos (con Cost) ----------
products_table = pd.DataFrame.from_records([
    ['Banano 1u', 0, 4, 89, 1, 0, 23, 0.30],
    ['Mandarina 1u', 0, 4, 40, 1, 0, 10, 0.25],
    ['Piña 100g', 0, 7, 50, 1, 0, 13, 0.6],
    ['Uvas 100g', 0, 7, 76, 1, 0, 17, 0.8],
    ['Chocolate 1 bar', 0, 4, 230, 3, 13, 25, 1.5],
    ['Queso Paipa 100g', 0, 8, 350, 28, 26, 2, 2.0],
    ['Quesillo 100g', 0, 8, 374, 18, 33, 1, 1.8],
    ['Pesto 100g', 0, 8, 303, 3, 30, 4, 1.2],
    ['Hummus 100g', 0, 8, 306, 7, 25, 11, 1.0],
    ['Pasta de berenjena 100g', 0, 4, 228, 1, 20, 8, 1.0],
    ['Batido de proteinas', 0, 5, 160, 30, 3, 5, 1.8],
    ['Hamburguesa vegetariana 1', 0, 5, 220, 21, 12, 3, 2.5],
    ['Hamburguesa vegetariana 2', 0, 12, 165, 16, 9, 2, 2.0],
    ['Huevo cocido 1', 0, 8, 155, 13, 11, 1, 0.4],
    ['Huevo frito 1', 0, 16, 196, 14, 15, 1, 0.45],
    ['Medio baguette', 0, 3, 274, 10, 0, 52, 0.9],
    ['Pan tajado 1 tajada', 0, 3, 97, 3, 1, 17, 0.25],
    ['Pizza de queso1u', 0, 3, 903, 36, 47, 81, 6.0],
    ['Pizza vegetariana 1u', 0, 3, 766, 26, 35, 85, 6.5],
    ['Leche de soya 200ml', 0, 1, 115, 8, 4, 11, 0.8],
    ['Leche de soya achocolatada 250ml', 0, 3, 160, 7, 6, 20, 1.0],
])
products_table.columns = ['Nombre', 'Min', 'Max', 'Calorias', 'Gram_Prot', 'Gram_Grasa', 'Gram_Carb', 'Cost']

# Copia inmutable de la "tabla de datos" como la quieres mantener
data_table = products_table.copy()

# extrae info para evaluación
cal_data = data_table[['Gram_Prot', 'Gram_Grasa', 'Gram_Carb']]
prot_data = list(cal_data['Gram_Prot'])
fat_data = list(cal_data['Gram_Grasa'])
carb_data = list(cal_data['Gram_Carb'])
cost_data = list(data_table['Cost'])
NUM_PRODUCTS = len(data_table)

# generador de individuos (respeta min/max de cada producto)
def n_per_product():
    return [random.randint(int(data_table.loc[i, 'Min']), int(data_table.loc[i, 'Max'])) for i in range(NUM_PRODUCTS)]

# evaluación multiobjetivo
def evaluate(individual):
    tot_prot = sum(x*y for x,y in zip(prot_data, individual))
    tot_fat = sum(x*y for x,y in zip(fat_data, individual))
    tot_carb = sum(x*y for x,y in zip(carb_data, individual))
    cals = prot_cal_p_gram * tot_prot + carb_cal_p_gram * tot_carb + fat_cal_p_gram * tot_fat
    cal_diff = abs(cals - total_calories)
    total_cost = sum(q * p for q,p in zip(individual, cost_data))
    return cal_diff, total_cost

# ---------- Evitar warning si se ejecuta varias veces en el mismo intérprete ----------
for cls_name in ("FitnessMin", "Individual"):
    if hasattr(creator, cls_name):
        delattr(creator, cls_name)

# ---------- Configuración DEAP ----------
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("n_per_product", n_per_product)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.n_per_product)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
max_allowed = int(data_table['Max'].max())
toolbox.register("mutate", tools.mutUniformInt, low=0, up=max_allowed, indpb=0.15)
toolbox.register("select", tools.selNSGA2)

# ---------- Parámetros ----------
POP_SIZE = 100
NGEN = 60
CXPB = 0.7
MUTPB = 0.3
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ---------- main (NSGA-II estilo manual, corregido) ----------
def main():
    pop = toolbox.population(n=POP_SIZE)
    # Evaluar población inicial
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = list(map(toolbox.evaluate, invalid_ind))
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    for gen in range(1, NGEN+1):
        # seleccionar padres (NSGA2)
        offspring = tools.selNSGA2(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]

        # cruza
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # mutación y clipping a min/max
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                for i in range(NUM_PRODUCTS):
                    if mutant[i] < int(data_table.loc[i, 'Min']):
                        mutant[i] = int(data_table.loc[i, 'Min'])
                    if mutant[i] > int(data_table.loc[i, 'Max']):
                        mutant[i] = int(data_table.loc[i, 'Max'])
                del mutant.fitness.values

        # evaluar nuevos individuos
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # reemplazo por NSGA-II
        pop = toolbox.select(pop + offspring, POP_SIZE)

        if gen % 10 == 0 or gen == 1:
            best = tools.selBest(pop, 1)[0]
            print(f"Gen {gen}: mejor individuo (cal_diff, cost) = {best.fitness.values}")

    # frente de Pareto (primer frente)
    pareto_front = tools.sortNondominated(pop, k=len(pop), first_front_only=False)[0]
    return pop, pareto_front

if __name__ == "__main__":
    pop, pareto_front = main()

    pareto_sorted = sorted(pareto_front, key=lambda ind: (ind.fitness.values[0], ind.fitness.values[1]))
    sample = pareto_sorted[:5] if len(pareto_sorted) >= 5 else pareto_sorted

    print("\nNúmero de soluciones en el frente de Pareto:", len(pareto_front))
    print("Muestreo de soluciones (cal_diff, cost):")
    for i, ind in enumerate(sample):
        print(i+1, ind.fitness.values)

    # tomar la mejor solución (la de menor diferencia calórica entre las muestreadas)
    if len(sample) == 0:
        print("No se encontraron soluciones en el frente de Pareto.")
    else:
        best = sample[0]
        quantities = list(best)  # lista de cantidades por producto

        # -------- Tabla de datos (original) -> sin SelectedQty ni columnas extras ----------
        print("\n--- Tabla de datos (sin columnas de selección) ---")
        print(data_table.to_string(index=False))

        # -------- Tabla de resultados (solo productos seleccionados > 0) ----------
        results = data_table.copy()
        results['SelectedQty'] = quantities
        results['TotalCalories'] = results['SelectedQty'] * results['Calorias']
        results['TotalCost'] = results['SelectedQty'] * results['Cost']

        selected_only = results[results['SelectedQty'] > 0].copy()
        # mostrar sólo las columnas solicitadas en la tabla de resultados
        selected_cols = ['Nombre', 'SelectedQty', 'TotalCalories', 'TotalCost']

        print("\n--- Resultados: Productos seleccionados (SelectedQty > 0) ---")
        if selected_only.empty:
            print("Ningún producto seleccionado (todas las cantidades son 0).")
        else:
            # Formatear TotalCost con 2 decimales en la impresión
            selected_only['TotalCost'] = selected_only['TotalCost'].map(lambda x: f"${x:.2f}")
            print(selected_only[selected_cols].to_string(index=False))

        # Totales
        total_calories_sel = results['TotalCalories'].sum()
        total_cost_sel = results['TotalCost'].sum()
        print(f"\nTotales de la solución seleccionada: Calorías seleccionadas = {total_calories_sel}, Costo total = ${total_cost_sel:.2f}")


calorias de proteina:  1050 calorias de carb:  1750 calorias de grasa:  700
gramos de proteina:  262.5 gramos de carb:  437.5 gramos de grasa:  77.77777777777777
Gen 1: mejor individuo (cal_diff, cost) = (5414.0, 53.85)
Gen 10: mejor individuo (cal_diff, cost) = (325.0, 22.55)
Gen 20: mejor individuo (cal_diff, cost) = (2.0, 24.3)
Gen 30: mejor individuo (cal_diff, cost) = (2.0, 19.45)
Gen 40: mejor individuo (cal_diff, cost) = (2.0, 19.45)
Gen 50: mejor individuo (cal_diff, cost) = (1.0, 13.4)
Gen 60: mejor individuo (cal_diff, cost) = (1.0, 13.4)

Número de soluciones en el frente de Pareto: 100
Muestreo de soluciones (cal_diff, cost):
1 (1.0, 13.4)
2 (1.0, 13.4)
3 (1.0, 13.4)
4 (1.0, 13.4)
5 (1.0, 13.4)

--- Tabla de datos (sin columnas de selección) ---
                          Nombre  Min  Max  Calorias  Gram_Prot  Gram_Grasa  Gram_Carb  Cost
                       Banano 1u    0    4        89          1           0         23  0.30
                    Mandarina 1u    0    4    